In [2]:
import warnings
import altair as alt
import pandas as pd
import warnings
warnings.simplefilter('ignore')

In [3]:
shots = pd.read_csv("https://data.cityofnewyork.us/api/views/833y-fsy8/rows.csv?accessType=DOWNLOAD",sep=",",index_col=0)

In [4]:
#shots.head()

In [5]:
# Integers to the MURDER T/F Column
shots['MURDER'] = (shots['STATISTICAL_MURDER_FLAG']==True).astype(int)
print("SHOTS:",shots.shape)
print(shots.describe())
print("20% of 27000 dataframe is reasonable for graphic exploration")
sho = shots.loc[shots['MURDER']==1]
print("SHO:",sho.shape)
#sho.head()

SHOTS: (27312, 21)
           PRECINCT  JURISDICTION_CODE    X_COORD_CD     Y_COORD_CD  \
count  27312.000000       27310.000000  2.731200e+04   27312.000000   
mean      65.635362           0.326913  1.009449e+06  208127.401608   
std       27.305705           0.737743  1.837783e+04   31886.377757   
min        1.000000           0.000000  9.149281e+05  125756.718750   
25%       44.000000           0.000000  1.000029e+06  182834.335938   
50%       68.000000           0.000000  1.007731e+06  194486.570312   
75%       81.000000           0.000000  1.016838e+06  239518.468750   
max      123.000000           2.000000  1.066815e+06  271127.687500   

           Latitude     Longitude        MURDER  
count  27302.000000  27302.000000  27312.000000  
mean      40.737892    -73.909051      0.192809  
std        0.087525      0.066272      0.394512  
min       40.511586    -74.249303      0.000000  
25%       40.668481    -73.943027      0.000000  
50%       40.700318    -73.915221      0.

In [6]:
# >5000 is too large for altair, so let's combine them;
#three groups:    date, hour, boro & month, hour, boro & year, hour, boro
#column of hour
sho['OT'] = sho['OCCUR_TIME'].astype(str)
sho['OT'] = sho['OT'].str[0:2].astype(int)
#colun of year, mo, & month/yr
sho['OD'] = sho['OCCUR_DATE'].astype(str)
sho['MO'] = sho['OD'].str[0:2] #step 1 month
sho['YR'] = sho['OD'].str[-5:] #step 2 /year
sho['OD'] = (sho['MO']+sho['YR'])  #step 3 combine
sho['YR'] = sho['YR'].str[-4:] # step 4 clean up that slash
sho['OD'] = pd.to_datetime(sho['OD']) #step 5 type for ordering
sho.head()

,OCCUR_DATE,OCCUR_TIME,BORO,LOC_OF_OCCUR_DESC,PRECINCT,JURISDICTION_CODE,LOC_CLASSFCTN_DESC,LOCATION_DESC,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,...,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat,MURDER,OT,OD,MO,YR
INCIDENT_KEY,,,,,,,,,,,,,,,,,,,,,
147998800,11/21/2015,03:56:00,QUEENS,NaN,108,0.0,NaN,NaN,True,NaN,...,1.007668e+06,209836.531250,40.742607,-73.915492,POINT (-73.91549174199997 40.74260663300004),1,3,2015-11-01,11,2015
58921844,02/19/2009,22:58:00,BRONX,NaN,47,0.0,NaN,NaN,True,25-44,...,1.024922e+06,262189.406250,40.886238,-73.852910,POINT (-73.85290950899997 40.88623791800006),1,22,2009-02-01,02,2009
219559682,10/21/2020,21:36:00,BROOKLYN,NaN,81,0.0,NaN,NaN,True,NaN,...,1.004234e+06,186461.703125,40.678457,-73.927952,POINT (-73.92795224099996 40.678456718000064),1,21,2020-10-01,10,2020
71662474,03/08/2010,19:41:00,BROOKLYN,NaN,81,0.0,NaN,NaN,True,NaN,...,1.002883e+06,192219.703125,40.694264,-73.932809,POINT (-73.93280863699994 40.694264056000065),1,19,2010-03-01,03,2010
73879554,07/27/2010,02:22:00,MANHATTAN,NaN,30,0.0,NaN,NaN,True,25-44,...,9.999674e+05,240325.562500,40.826308,-73.943210,POINT (-73.94320965699995 40.82630793100003),1,2,2010-07-01,07,2010


In [7]:
#rekey
sho = sho.reset_index()
#first group by month, hour, and borough
sh = sho.groupby(['OD','BORO','OT'])['MURDER'].aggregate('sum')
sh.head()

OD          BORO   OT
2006-01-01  BRONX  1     1
                   5     1
                   7     1
                   20    1
                   21    4
Name: MURDER, dtype: int64

In [8]:
#second group by year, hour, and borough
shd = pd.DataFrame(sho.groupby(['YR','BORO','OT'])['MURDER'].aggregate('sum'))
shd.head()

MURDER
YR   BORO  OT        
2006 BRONX 0        6
           1        9
           2        6
           3       11
           4       14

In [9]:
# altair doesnt support hierarchical indices, so flatten them
shd = shd.reset_index()
sh = sh.reset_index()

In [18]:
print("Overview of Murders by Month, and Borough")
#print("Overview of Murders By Date (Month) and Borough:")
#print("Zoom in for granular detail by month...")
#implement filtering using dynamic
selection = alt.selection(type="multi", fields=['BORO'])

#Create a container for our charts
base= alt.Chart(sh).properties(width=1000,height=500)

#overview chart
overview = alt.Chart(sh).mark_bar().encode(
    y= alt.Y("BORO",title=""),
    x= alt.X("sum(MURDER)",title="Murder Count (Click to Zoom)"),
    color=alt.condition(selection,alt.value("#880000"),alt.value("lightgrey")
    )).add_selection(selection).properties(height=500,width=500)

#detail chart
detailed = alt.Chart(sh).mark_line().encode(
    x= alt.X("OD",title="Murders By Month (scroll to zoom)"),
    y=alt.Y("sum(MURDER)",title="Murder Count",axis=alt.Axis(orient='right')),
    color = alt.value('#880000'),
    tooltip=['OD','BORO','sum(MURDER)']
    #,opacity = alt.condition(selection,alt.value(1),alt.value(.2))

).transform_filter(selection).properties(height=500,width=500).interactive()
overview | detailed

Overview of Murders by Month, and Borough


alt.HConcatChart(...)

In [17]:
print("Overview of Murders by Time of Day, and Borough")
from pandas.core.series import Axis
#print("Overview of Murders by Time of Day and Borough (2006-2022)")
#implement filtering using dynamic on hour of day
selection = alt.selection(type="multi", fields=['BORO'])

#Create a container for our charts
base= alt.Chart(sh).properties(width=1000,height=500)

#overview chart
overview = alt.Chart(sh).mark_bar().encode(
    y = alt.Y("BORO",  title=""),
    x = alt.X("sum(MURDER)", title="Murder Count (Click To View)"),
    color=alt.condition(selection,alt.value("#880020"),alt.value("lightgrey")
    )).add_selection(selection).properties(height=500,width=500)

#detail chart
detailed = alt.Chart(sh).mark_line().encode(
    x = alt.X("OT", title="Hours of the Day (Scroll to Zoom)"),
    y = alt.Y("sum(MURDER)", title="Murder Count ",axis=alt.Axis(orient='right')),
    #color=  alt.Color('OT',scale=alt.Scale(scheme='dark2')),
    color=alt.value('#880000'),
    tooltip=['BORO','OT','sum(MURDER)']
).transform_filter(selection).properties(height=500,width=500).interactive()
overview | detailed

Overview of Murders by Time of Day, and Borough


alt.HConcatChart(...)

In [16]:
print("Overview of Murders by Time of Day, Year, and Borough")
#implement filtering using dynamic on hour of day
selection = alt.selection(type="multi", fields=['BORO'])
selection2 = alt.selection(type="multi", fields=['YR'])

#Create a container for our charts
chart = alt.Chart(shd).properties(width=700,height=300)

#overview chart
overview = alt.Chart(shd).mark_bar().encode(
    y="BORO",
    x="sum(MURDER)",
    color=alt.condition(selection,alt.value("#880020"),alt.value("lightgrey")
    )).add_selection(selection).properties(height=500,width=500)

#detail chart
detailed = alt.Chart(shd).mark_line().encode(
    x="OT",
    y="sum(MURDER)",
    color=  alt.Color('YR',scale=alt.Scale(scheme='dark2')),
    #color=alt.value('red'),
    tooltip=['BORO','OT','sum(MURDER)','YR'],
    #opacity=alt.condition(selection2,alt.value(1),alt.value(.2))
).transform_filter(selection).properties(height=500,width=500).interactive()
overview | detailed

#detailed.save('details.html')


Overview of Murders by Time of Day and Borough


alt.HConcatChart(...)

In [ ]:
12


12